In [1]:

import pandas as pd
import latentscope as ls
import h5py
# !pip install datasets
from datasets import load_dataset

In [2]:
# TODO: show downloading sample from fineweb

In [3]:
with h5py.File("/Users/enjalot/latent-scope-data/fineweb-edu-10k/saes/sae-001.h5", "r") as f:
    print(f.keys())
    top_acts = f.get("top_acts")[:]
    top_indices = f.get("top_indices")[:]
    print(top_acts.shape)
    print(top_indices.shape)


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/Users/enjalot/latent-scope-data/fineweb-edu-10k/saes/sae-001.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [4]:
top_acts[0]

NameError: name 'top_acts' is not defined

In [12]:
top_indices[0]

array([ 8944,  7583, 16225, 14943, 11176, 23917,  4919,  4941,  4000,
       10367,  5983,  7627,  9398, 16267, 11963, 16708,  7518, 18204,
        9535,  2724, 18167,  2228, 19213, 18010, 22989,  5432, 10935,
       22337,  4004, 11119,  7774, 17695, 10567, 21551, 23250,  1460,
       22268,  9566, 18271, 21744, 21539, 10463, 20740,  5340, 10539,
        9217, 15487,  7070, 22817, 17483, 24236, 18312, 20104, 12048,
        3322,    38,  4430, 13258,  3523,  1725, 20912, 15362,  8929,
         519])

In [5]:
# experiment with loading a saved UMAP model for the SAE top activations that we can use to map any dataset
from huggingface_hub import hf_hub_download
import pickle
import json

# Download the UMAP model from HuggingFace Hub
model_path = hf_hub_download(
    repo_id="enjalot/sae-nomic-text-v1.5-FineWeb-edu-100BT",
    filename="umap/64_32/umap-top10.pkl"
)
# Download the metadata JSON file
metadata_path = hf_hub_download(
    repo_id="enjalot/sae-nomic-text-v1.5-FineWeb-edu-100BT",
    filename="umap/64_32/umap-top10-metadata.json"
)


# Load the UMAP model
with open(model_path, 'rb') as f:
    umap_model = pickle.load(f)


# Load the metadata
with open(metadata_path, 'r') as f:
    umap_metadata = json.load(f)

print("UMAP model loaded:", umap_model)
print("metadata", umap_metadata)


umap-top10.pkl:   0%|          | 0.00/287M [00:00<?, ?B/s]

umap/64_32/umap-top10-metadata.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

: 

In [ ]:
# TODO: load some embeddings and fit this
xy = umap_model.transform(embeddings)

In [ ]:
min_values = np.array([umap_metadata['bounds']["x_min"], umap_metadata['bounds']["y_min"]])
max_values = np.array([umap_metadata['bounds']["x_max"], umap_metadata['bounds']["y_max"]])
normalized_points = (xy - min_values) / (max_values - min_values)
normalized_points = 2 * normalized_points - 1
# you can now get the x,y coordinates that are scaled to the original umap x,y
# this means any dataset projected in this way will overlap in the exact same space as the original SAE top activations

# results_df["x"] = normalized_points[:, 0]
# results_df["y"] = normalized_points[:, 1]
# results_df.head()

In [9]:
# Load the nomic text embed model
from transformers import AutoTokenizer, AutoModel
import torch

model_name = "nomic-ai/nomic-embed-text-v1.5"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

def mean_pooling(token_embeddings, attention_mask):
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


def get_hidden_states(text, model, tokenizer):
    # Tokenize the text and move to GPU if available
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Get hidden states
    with torch.no_grad():
        outputs = model(**inputs)
        
    # Get the last hidden state
    embedding = mean_pooling(outputs[0], inputs["attention_mask"])
    return { "hidden_states": outputs[0].cpu().numpy(), "embedding": embedding.cpu().numpy() }

# Example usage:
text = ["Where do dads keep their dad jokes? In a dad-a-base"]
hs = get_hidden_states(text, model, tokenizer)
pertoken = hs["hidden_states"]
embedding = hs["embedding"]


<All keys matched successfully>


In [10]:
print(pertoken.shape)
print(embedding.shape)


(1, 18, 768)
(1, 768)


In [26]:
from latentsae.sae import Sae

# Use same model_id and k_expansion as in sae.py
model_id = "enjalot/sae-nomic-text-v1.5-FineWeb-edu-100BT"
k_expansion = "64_32"

device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device)

# Load SAE model
sae_model = Sae.load_from_hub(model_id, k_expansion)


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Dropping extra args {'signed': False}


In [29]:
# Convert numpy arrays to torch tensors and move to device
embedding_tensor = torch.from_numpy(embedding).float().to(device)
pertoken_tensor = torch.from_numpy(pertoken).float().to(device)

# Reshape pertoken to 2D (combine batch and sequence dims)
batch_size, seq_len, hidden_dim = pertoken_tensor.shape
pertoken_2d = pertoken_tensor.reshape(-1, hidden_dim)

print("Encoding embeddings...")
embedding_features = sae_model.encode(embedding_tensor)

print("\nEncoding per-token hidden states...")
pertoken_features = sae_model.encode(pertoken_2d)

# Print shapes and sample activations
print("\nEmbedding features:")
print("Top activations shape:", embedding_features.top_acts.shape)
print("Top indices shape:", embedding_features.top_indices.shape)
# print("Sample top activations:", embedding_features.top_acts[0][:5])
print("Sample top indices:", embedding_features.top_indices[0][:10])


Encoding embeddings...

Encoding per-token hidden states...

Embedding features:
Top activations shape: torch.Size([1, 64])
Top indices shape: torch.Size([1, 64])
Sample top indices: tensor([ 6328,  6939, 24120,  3945,  5750,  6073, 13056,  9757, 10978, 22978])


In [30]:
# Get the top activations and indices from pertoken features
pertoken_acts = pertoken_features.top_acts
pertoken_inds = pertoken_features.top_indices

# Sort indices by activation values
sorted_acts, sorted_idx = torch.sort(pertoken_acts, dim=1, descending=True)

# Use the sorted indices to get the corresponding feature indices
sorted_feature_inds = torch.gather(pertoken_inds, 1, sorted_idx)

print("\nPer-token features sorted by activation:")
print("Shape:", sorted_feature_inds.shape)
print("Top 10 most activated indices for first token:", sorted_feature_inds[0,:10].cpu().numpy())



Per-token features sorted by activation:
Shape: torch.Size([18, 64])
Top 10 most activated indices for first token: [ 6939  6328  3945 13923 24120 19936 22978 13056 16387  5587]


In [31]:
# pip install -e .
from latentsae.widgets import TopK, Embedding

In [33]:
import pandas as pd
sae_features = pd.read_parquet("https://enjalot.github.io/latent-taxonomy/models/NOMIC_FWEDU_25k/features.parquet")

In [34]:
features = sae_features.to_dict(orient='records')

In [35]:
def get_maxk(directions, k=32):
    # Flatten the top_acts and top_indices to find the global top k
    flat_acts = directions.top_acts.flatten()
    flat_indices = directions.top_indices.flatten()

    # Create a dictionary to keep track of the highest activation for each index
    max_acts = {}
    
    for act, idx in zip(flat_acts, flat_indices):
        if idx.item() not in max_acts or act.item() > max_acts[idx.item()]:
            max_acts[idx.item()] = act.item()
    
    # Convert the dictionary back to tensors
    flat_acts = torch.tensor(list(max_acts.values()))
    flat_indices = torch.tensor(list(max_acts.keys()))
    
    # Get the top k activations and their corresponding indices
    topk_acts, topk_indices = torch.topk(flat_acts, k)
    
    # Map the flat indices back to the original indices
    original_indices = flat_indices[topk_indices]
    
    topk = {
        "top_acts": topk_acts,
        "top_indices": original_indices
    }
    return topk

In [44]:
def get_token_direction(directions, idx):
    return { "top_indices": directions.top_indices[idx], "top_acts": directions.top_acts[idx] }


In [47]:
maxk = get_maxk(pertoken_features, k=32)

In [49]:
TopK(data=maxk, n=30, features=features)

TopK(data={'top_acts': [12.723018646240234, 11.715324401855469, 8.681981086730957, 8.520795822143555, 8.355340…

In [45]:
TopK(data=get_token_direction(embedding_features, 0), n=10, features=features)

TopK(data={'top_indices': [6328, 6939, 24120, 3945, 5750, 6073, 13056, 9757, 10978, 22978, 20602, 13923, 22022…